Nota: Este código ha sido escrito con la ayuda de GitHub Copilot, principalmente las sugerencias de autocompletar. Siendo esta una módficación de apartado_1.ipynb, solo se incluirá comentarios para las modificaciones frente a la original.

In [72]:
import numpy as np
from numba import jit
import pandas as pd
import matplotlib.pyplot as plt

Se reduce el número de partículas a 16.

In [73]:
N = 16
L = 10.0
h = 0.002

v_0 = 5

epsilon = 1
sigma = 1
m = 1
k=1

skip = 5

Time = 50
time = np.arange(0, Time, h)

f = open(f"datos_apartado_3/posiciones_{v_0}.txt", "w")
f_energia = open(f"datos_apartado_3/energias_{v_0}.txt", "w")
f_velocidad = open(f"datos_apartado_3/velocidades_{v_0}.txt", "w")

Se sigue empleando la misma notación, los únicos cambios introducidos seria:
- En las condiciones de contorno, se ha añadido una variable bool, que devuelve si la partícula considerada en el bucle, ha cruzado la frontera. Además de quitar la aleatoriedad en las posiciones iniciales.
- En el algoritmo de verlet se ha introducido una variable, que va sumando el momento transferido en dicho instante de tiempo.

In [74]:
@jit(nopython=True)
def init_cond():
    r = np.zeros((N, 2))
    v =np.zeros((N, 2))
    for i in range(N):
        r[i] = np.array([i%4*2.5 +1, i//4*2.5 +1])
        theta = np.random.rand()*2*np.pi
        v[i] = v_0 * np.array([np.sin(theta), np.cos(theta)])
    return r, v

@jit(nopython=True)
def cond_contorno(r):
    Cond = False
    if(r[0] > L):
        r[0] = r[0] - L
        Cond = True
    if(r[0] < 0):
        r[0] = r[0] + L
        Cond = True
    if(r[1] > L):
        r[1] = r[1] - L
        Cond = True
    if(r[1] < 0):
        r[1] = r[1] + L
        Cond = True
    return r, Cond

@jit(nopython=True)
def cond_contorno_distancia(r):
    if(r[0] > L/2):
        r[0] = r[0] - L
    elif(r[0] < -L/2):
        r[0] = r[0] + L
    if(r[1] > L/2):
        r[1] = r[1] - L
    elif(r[1] < -L/2):
        r[1] = r[1] + L
    return r

@jit(nopython=True)
def compute_distance(r):
    R = np.zeros((N, N, 2))
    for i in range(0, N-1):
        for j in range(i+1, N):
            R[i, j] = r[j]- r[i]
            R[i, j] = cond_contorno_distancia(R[i, j])
            R[j, i] = -R[i, j]
    return R

@jit(nopython=True)
def lennard_jones(r):
    R = compute_distance(r)
    acc = np.zeros((N, 2))
    for i in range(N):
        for j in range(N):
            if(i!=j):
                norm  = np.linalg.norm(R[i, j])
                if (norm < 3):
                    acc[i] = acc[i] + 4*R[i, j]* epsilon * (6*np.power((sigma/norm), 5) - 12*np.power((sigma/norm), 11))/(norm*m)
    return acc, R

@jit(nopython=True)
def verlet_algorithm(r, v, a):
    w = np.zeros((N, 2))
    p = 0
    for i in range(N):
        r[i] = r[i] + h*v[i] + h*h*a[i]/2
        r[i], Cond = cond_contorno(r[i])
        if(Cond):
            p = p+2*m*np.linalg.norm(v[i])
        w[i] = v[i] + h*a[i]/2
    a, R = lennard_jones(r)
    for i in range(N):
        v[i] = w[i] + h*a[i]/2
    return r, v, a, R, p

@jit(nopython=True)
def compute_energy(v, R):
    T = 0
    V = 0
    for i in range(N):
        T = T + 0.5*m*np.linalg.norm(v[i])**2
        for j in range(N):
            if(i!=j):
                norm  = np.linalg.norm(R[i, j])
                V = V + 4*epsilon * (np.power((sigma/norm), 12) - np.power((sigma/norm), 6))
    return T, V

@jit(nopython=True)
def compute_average_speed(v):
    v_prom = 0
    for i in range(N):
        v_prom = v_prom + np.linalg.norm(v[i])
    return v_prom/N


def write_vector(r, f):
    for i in range(N):
        f.write(f"{r[i, 0]}, {r[i, 1]}\n")
    f.write(f"\n")

def write_velocity(v, f):
    for i in range(N):
        f.write(f"{v[i, 0]}, {v[i, 1]}, {np.linalg.norm(v[i])}\n")

Se ha añadido el momento transderido a la pared, sumando a todos los instantes de tiempo, tenemos el momento total transferido. La fuerza ejercida sería el momento total transferido entre el tiempo total considerado. Que en este caso solo se ha empezado a tomar en cuenta cuando el sistema se he relajado.

Para hallar la presión, dividimos la fuerza por el área, sin embargo en este caso al estar en 2 dimensiones, sería la longitud del cuadrado.

In [75]:
T, V = np.zeros(len(time)), np.zeros(len(time))
momento = 0

r, v = init_cond()
write_vector(r, f)
a, R = lennard_jones(r)
T[0], V[0] = compute_energy(v, R)
f_energia.write(f"{T[0]}, {V[0]}\n")

for i in range(1, len(time)):
    r, v, a, R, p = verlet_algorithm(r, v, a)
    T[i], V[i] = compute_energy(v, R)
    if (i%skip == 0):
        write_vector(r, f)
    if(i >= int(20/h)):
        momento = momento + p
        write_velocity(v, f_velocidad)
    f_energia.write(f"{T[i]}, {V[i]}\n")

F = momento/(Time-20)

f.close()
f_velocidad.close()
f_energia.close()

Se lee el archivo de las velocidades para hallar la temperatura del sistema, y se añade al archivo la velocidad inicial, la temperatura y la presión del sistema.

In [76]:
velocidad = pd.read_csv(f'datos_apartado_3/velocidades_{v_0}.txt', delimiter=',', index_col=False, header=0 ,names=['vx', 'vy', 'v'])

Temp = m/(2*k)*np.mean(velocidad.v**2)

f_pressure = open(f"datos_apartado_3/pressure.txt", "a")
f_pressure.write(f"{v_0}, {Temp}, {F/L} \n")
f_pressure.close()